In [32]:
%load_ext autoreload
%autoreload 2

from pysat.formula import CNF
from compiler.entrypoint import Max3satQaoaCompiler

formula = CNF(from_file="./benchmarks/uf20-01.cnf")

print(formula)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CNF(from_string='c This Formular is generated by mcnf\nc\nc    horn? no\nc    forced? no\nc    mixed sat? no\nc    clause length = 3\nc\np cnf 20 91\n4 -18 19 0\n3 18 -5 0\n-5 -8 -15 0\n-20 7 -16 0\n10 -13 -7 0\n-12 -9 17 0\n17 19 5 0\n-16 9 15 0\n11 -5 -14 0\n18 -10 13 0\n-3 11 12 0\n-6 -17 -8 0\n-18 14 1 0\n-19 -15 10 0\n12 18 -19 0\n-8 4 7 0\n-8 -9 4 0\n7 17 -15 0\n12 -7 -14 0\n-10 -11 8 0\n2 -15 -11 0\n9 6 1 0\n-11 20 -17 0\n9 -15 13 0\n12 -7 -17 0\n-18 -2 20 0\n20 12 4 0\n19 11 14 0\n-16 18 -4 0\n-1 -17 -19 0\n-13 15 10 0\n-12 -14 -13 0\n12 -14 -7 0\n-7 16 10 0\n6 10 7 0\n20 14 -16 0\n-19 17 11 0\n-7 1 -20 0\n-5 12 15 0\n-4 -9 -13 0\n12 -11 -7 0\n-5 19 -8 0\n1 16 17 0\n20 -14 -15 0\n13 -4 10 0\n14 7 10 0\n-5 9 20 0\n10 1 -19 0\n-16 -15 -1 0\n16 3 -11 0\n-15 -10 4 0\n4 -15 -3 0\n-10 -16 11 0\n-8 12 -5 0\n14 -6 12 0\n1 6 11 0\n-13 -5 -1 0\n-7 -2 12 0\n1 -20 19 0\n-2 -13 -8 0\n15 18 4 0\n-11 14 9 

In [34]:
from nac.config import FPQAConfig

config = {
    "CCZ_GATE_FIDELITY": 0.98
}
fpqa_config = FPQAConfig(config)
print(fpqa_config.CCZ_GATE_FIDELITY)

compiler = Max3satQaoaCompiler(formula, fpqa_config)
program = compiler.compile_single_layer()
fidelity, duration = program.avg_fidelity(), program.duration()
print("Number of Instructions: ", len(program.instructions), ", Fidelity: ", fidelity, ", Duration ", duration)
print(program.count_ops())

0.98
Number of Instructions:  1388 , Fidelity:  0.014237571795551816 , Duration  10455.943562714388
{'u3': 633, 'cz': 194, 'ccz': 124}


In [13]:
from utils.hamiltonians import Max3satHamiltonian
from utils.qaoa import QAOA
from qiskit import transpile

basis_gates = ["u3", "id", "cz"]
hamiltonian = Max3satHamiltonian(formula=formula)
qaoa = QAOA(hamiltonian)
circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(1)
bound_circuit = circuit.assign_parameters({cost_params: [0.2512], mixer_params: [0.1235]})
transpiled_circuit = transpile(bound_circuit, basis_gates=basis_gates, optimization_level=3)
print(bound_circuit.count_ops())
ops = transpiled_circuit.count_ops()
print("Fidelity: ", (0.999 ** ops["u3"]) * (0.995 ** ops["cz"]))
print(ops)


OrderedDict([('cx', 590), ('rz', 231), ('h', 20), ('rx', 20)])
Fidelity:  0.06394125013650653
OrderedDict([('u3', 544), ('cz', 440)])


In [19]:
program.write("test.wqasm")

# Weaver Evaluation

In [36]:
import os
import time
import pandas as pd
from utils.sat_utils import get_color_map

data = []
columns = [
    "name", 
    "num_variables", 
    "num_clauses", 
    "num_colors", 
    "compilation_time (seconds)", 
    "execution_time (microseconds)",
    "eps (fidelity)", 
    "#u3", 
    "#cz",
    "#ccz", 
    "ccz_fidelity", 
    "fpqa_config"
]
ccz_fidelities = [0.9775, 0.98, 0.9825, 0.985, 0.9875, 0.99, 0.9925, 0.995, 0.9975]
benchmarks = list(filter(lambda f: f.endswith(".cnf"), os.listdir("./benchmarks")))
num_benchmarks = len(benchmarks)

for index, filename in enumerate(benchmarks):
    fpqa_config = FPQAConfig({})
    print(f"Compiling {filename} ({index + 1}/{num_benchmarks})...")
    if not filename.endswith(".cnf"):
        continue
    formula = CNF(from_file=f"./benchmarks/{filename}")
    num_colors, color_map = get_color_map(formula)
    start_time = time.time()
    compiler = Max3satQaoaCompiler(formula, fpqa_config)
    program = compiler.compile_single_layer()
    execution_time = program.duration()
    gates = program.count_ops()
    end_time = time.time()
    compilation_time = end_time - start_time
    for ccz_fidelity in ccz_fidelities:
        program.fpqa.config.CCZ_GATE_FIDELITY = ccz_fidelity
        fidelity = program.avg_fidelity()
        row = [
            filename, 
            str(formula.nv), 
            str(len(formula.clauses)), 
            str(num_colors), 
            str(compilation_time), 
            str(execution_time), 
            str(fidelity), 
            str(gates["u3"]), 
            str(gates["cz"]), 
            str(gates["ccz"]), 
            str(ccz_fidelity), 
            fpqa_config.to_string()
        ]
        data.append(row)
        
df = pd.DataFrame(data, columns=columns)
df.to_csv("results/fpqa-1.csv")

Compiling uuf175-02.cnf (1/105)...
Compiling uuf250-09.cnf (2/105)...
Compiling uf20-07.cnf (3/105)...
Compiling uuf50-07.cnf (4/105)...
Compiling uuf50-02.cnf (5/105)...
Compiling uuf100-05.cnf (6/105)...
Compiling uuf150-01.cnf (7/105)...
Compiling uuf175-08.cnf (8/105)...
Compiling uuf175-07.cnf (9/105)...
Compiling uf20-06.cnf (10/105)...
Compiling uuf125-02.cnf (11/105)...
Compiling uuf200-03.cnf (12/105)...
Compiling uuf225-09.cnf (13/105)...
Compiling uuf200-04.cnf (14/105)...
Compiling uf20-03.cnf (15/105)...
Compiling uuf75-06.cnf (16/105)...
Compiling uuf250-02.cnf (17/105)...
Compiling uuf150-010.cnf (18/105)...
Compiling uuf175-04.cnf (19/105)...
Compiling uuf125-04.cnf (20/105)...
Compiling uuf150-03.cnf (21/105)...
Compiling f600.cnf (22/105)...
Compiling uuf175-09.cnf (23/105)...
Compiling uuf75-03.cnf (24/105)...
Compiling uuf50-06.cnf (25/105)...
Compiling uf20-09.cnf (26/105)...
Compiling uuf175-05.cnf (27/105)...
Compiling 2d0c041c0fe72dc32527bfbf34f63e61-170223547.c

In [37]:
print(df)

               name num_variables num_clauses num_colors  \
0     uuf175-02.cnf           175         753         24   
1     uuf175-02.cnf           175         753         24   
2     uuf175-02.cnf           175         753         24   
3     uuf175-02.cnf           175         753         24   
4     uuf175-02.cnf           175         753         24   
..              ...           ...         ...        ...   
940  uuf175-010.cnf           175         753         23   
941  uuf175-010.cnf           175         753         23   
942  uuf175-010.cnf           175         753         23   
943  uuf175-010.cnf           175         753         23   
944  uuf175-010.cnf           175         753         23   

    compilation_time (seconds) execution_time (microseconds)  \
0           3.2473678588867188             121019.6155262156   
1           3.2473678588867188             121019.6155262156   
2           3.2473678588867188             121019.6155262156   
3           3.247367858